In [56]:
import os
import numpy as np
import pandas as pd
#import xgboost as xgb
import matplotlib.pyplot as plt
#from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score,classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.base import BaseEstimator
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from contextualized.easy import ContextualizedRegressor

# Time series decomposition
from statsmodels.tsa.seasonal import seasonal_decompose

# Chart drawing
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Mute sklearn warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

# Show charts when running kernel
init_notebook_mode(connected=True)

# Change default background color for all visualizations
layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(250,250,250,0.8)')
fig = go.Figure(layout=layout)
templated_fig = pio.to_templated(fig)
pio.templates['my_template'] = templated_fig.layout.template
pio.templates.default = 'my_template'

In [57]:

file_path = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/scripts/dati_with_context_f.csv' # cleaned data
df = pd.read_csv(file_path)
df = df.dropna(subset=['relative_quarterly_return'])

df.info()
df.tail()

<class 'pandas.core.frame.DataFrame'>
Index: 11723 entries, 1 to 11812
Data columns (total 64 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   tic                             11723 non-null  object 
 1   datadate                        11723 non-null  object 
 2   roa                             11692 non-null  float64
 3   roe                             11563 non-null  float64
 4   pe_inc                          11199 non-null  float64
 5   divyield                        9443 non-null   float64
 6   pcf                             11692 non-null  float64
 7   roce                            11586 non-null  float64
 8   gpm                             11711 non-null  float64
 9   evm                             11636 non-null  float64
 10  npm                             11712 non-null  float64
 11  PEG_trailing                    8110 non-null   float64
 12  de_ratio                        11708

,tic,datadate,roa,roe,pe_inc,divyield,pcf,roce,gpm,evm,...,r_roe,r_eg,divyield_Median,PEG_trailing_Median,pe_inc_Median,ptb_Median,evm_Median,roce_Median,roe_Median,bm_Median
11808,ZTS,2018-12-31,0.230,0.594,32.160,0.696,25.204,0.250,0.708,21.308,...,3.2636,4.3514,1.33,-0.101,-7.090,3.183,-4.273,-0.413,-0.531,0.227
11809,ZTS,2019-03-31,0.239,0.577,35.332,0.649,27.021,0.252,0.715,21.631,...,-3.5981,-5.1201,1.24,-0.079,-2.895,3.745,-2.919,-0.446,-0.452,0.308
11810,ZTS,2019-06-30,0.233,0.539,44.671,0.519,32.794,0.241,0.722,22.510,...,2.6492,1.1047,1.21,-0.115,-7.734,3.796,-3.920,-0.411,-0.512,0.251
11811,ZTS,2019-09-30,0.228,0.538,39.907,0.544,31.944,0.234,0.728,23.287,...,3.8375,0.8189,1.22,-0.092,-6.526,3.278,-3.883,-0.416,-0.509,0.258
11812,ZTS,2019-12-31,0.226,0.517,42.839,0.600,35.251,0.235,0.736,27.557,...,-1.9816,0.6485,1.10,-0.088,-8.014,3.910,-3.433,-0.447,-0.545,0.285


In [58]:
print(df.columns.tolist())
print(df.notna().sum().sort_values())



['tic', 'datadate', 'roa', 'roe', 'pe_inc', 'divyield', 'pcf', 'roce', 'gpm', 'evm', 'npm', 'PEG_trailing', 'de_ratio', 'ptb', 'market_cap', 'prccq_change', 'capxy_change', 'oancfy_change', 'actq_change', 'lctq_change', 'roa_change', 'roe_change', 'pe_inc_change', 'divyield_change', 'pcf_change', 'roce_change', 'gpm_change', 'evm_change', 'npm_change', 'PEG_trailing_change', 'de_ratio_change', 'ptb_change', 'next_prccq_change', 'relative_quarterly_return', 'next_relative_quarterly_return', 'volatility_score', 'FFI10_desc', 'mktrf', 'smb', 'hml', 'rmw', 'cma', 'rf', 'umd', 'h_1_x', 'h_3_x', 'h_12_x', 'h_1_y', 'h_3_y', 'h_12_y', 'r_f', 'r_mkt', 'r_me', 'r_ia', 'r_roe', 'r_eg', 'divyield_Median', 'PEG_trailing_Median', 'pe_inc_Median', 'ptb_Median', 'evm_Median', 'roce_Median', 'roe_Median', 'bm_Median']
PEG_trailing_change     7371
PEG_trailing            8110
oancfy_change           8131
capxy_change            8602
lctq_change             9189
                       ...  
roe_Median   

In [59]:


df = df.loc[:, df.count() >= 11000]

# Drop rows with missing values
df = df.dropna()

# Display dataframe info
df.info()

print(df.columns.tolist())


<class 'pandas.core.frame.DataFrame'>
Index: 10509 entries, 2 to 11812
Data columns (total 56 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   tic                             10509 non-null  object 
 1   datadate                        10509 non-null  object 
 2   roa                             10509 non-null  float64
 3   roe                             10509 non-null  float64
 4   pe_inc                          10509 non-null  float64
 5   pcf                             10509 non-null  float64
 6   roce                            10509 non-null  float64
 7   gpm                             10509 non-null  float64
 8   evm                             10509 non-null  float64
 9   npm                             10509 non-null  float64
 10  de_ratio                        10509 non-null  float64
 11  ptb                             10509 non-null  float64
 12  market_cap                      10509

In [60]:
# Find the unique number of stocks in the dataframe
unique_gvkeys = df['tic'].unique()

# Print the result
print(f"Number of unique gvkeys: {len(unique_gvkeys)}")
print(unique_gvkeys)


Number of unique gvkeys: 76
['AAPL' 'ABT' 'ADM' 'AFL' 'AIG' 'AMD' 'AMT' 'AMZN' 'APD' 'BA' 'BAX' 'BMY'
 'BRK.B' 'CAT' 'CL' 'CME' 'COP' 'CTSH' 'CVS' 'CVX' 'DIS' 'ECL' 'EQIX'
 'EXC' 'EXPE' 'F' 'FIS' 'GE' 'GOOGL' 'GS' 'HCA' 'IBM' 'INTC' 'JNJ' 'JPM'
 'KMB' 'KO' 'LLY' 'LMT' 'LUV' 'MA' 'MCD' 'META' 'MMC' 'MMM' 'MRK' 'MS'
 'MSFT' 'NFLX' 'NOC' 'OXY' 'PFE' 'PG' 'PGR' 'PPL' 'PYPL' 'QCOM' 'REGN'
 'RJF' 'RTX' 'SBUX' 'SCHW' 'SPGI' 'T' 'TMO' 'TSLA' 'TSN' 'UNH' 'UPS' 'V'
 'VZ' 'WBA' 'WDC' 'WFC' 'XOM' 'ZTS']


In [61]:
df = df.dropna()
df = df.sort_values(by=['datadate'])

unique_gvkeys = df['tic'].unique()

# Print the result
print(f"Number of unique gvkeys: {len(unique_gvkeys)}")

df.tail()

Number of unique gvkeys: 76


,tic,datadate,roa,roe,pe_inc,pcf,roce,gpm,evm,npm,...,r_roe,r_eg,divyield_Median,PEG_trailing_Median,pe_inc_Median,ptb_Median,evm_Median,roce_Median,roe_Median,bm_Median
876,AIG,2019-12-31,0.023,0.054,11.273,-37.158,0.066,0.236,7.292,0.067,...,-1.9816,0.6485,2.2,0.616,13.611,1.387,10.016,0.102,0.097,0.770
5972,JPM,2019-12-31,0.027,0.153,10.831,59.034,0.083,0.773,13.715,0.256,...,-1.9816,0.6485,2.2,0.616,13.611,1.387,10.016,0.102,0.097,0.770
5772,JNJ,2019-12-31,0.181,0.228,23.886,15.131,0.237,0.751,14.671,0.184,...,-1.9816,0.6485,1.1,-0.088,-8.014,3.910,-3.433,-0.447,-0.545,0.285
1932,BAX,2019-12-31,0.161,0.125,43.249,20.255,0.154,0.502,17.713,0.088,...,-1.9816,0.6485,2.2,0.616,13.611,1.387,10.016,0.102,0.097,0.770
11812,ZTS,2019-12-31,0.226,0.517,42.839,35.251,0.235,0.736,27.557,0.239,...,-1.9816,0.6485,1.1,-0.088,-8.014,3.910,-3.433,-0.447,-0.545,0.285


In [62]:
tickers_to_drop = ['AMT', 'CTSH', 'GOOGL', 'META', 'PYPL', 'RJF', 'TSLA', 'ZTS']

#df = df[~df['tic'].isin(tickers_to_drop)]

In [63]:
train_start_date = '1900-03-31'
valid_start_date = '2008-03-31'
test_start_date = '2013-03-31'
end_date = '2026-12-30'

df = df.sort_values(by=['datadate'])

train_df = df[(df['datadate'] >= train_start_date) & (df['datadate'] < valid_start_date)]
valid_df = df[(df['datadate'] >= valid_start_date) & (df['datadate'] < test_start_date)]
test_df = df[(df['datadate'] >= test_start_date) & (df['datadate'] <= end_date)]

test_df = test_df.sort_values(by=['datadate', 'next_prccq_change'], ascending=[True, False])
test_df.tail()


,tic,datadate,roa,roe,pe_inc,pcf,roce,gpm,evm,npm,...,r_roe,r_eg,divyield_Median,PEG_trailing_Median,pe_inc_Median,ptb_Median,evm_Median,roce_Median,roe_Median,bm_Median
4330,EXPE,2019-12-31,0.095,0.138,26.159,4.793,0.111,0.828,11.237,0.047,...,-1.9816,0.6485,1.37,0.306,1.471,3.459,12.578,0.043,0.004,0.289
4530,F,2019-12-31,0.060,0.001,306.600,1.537,0.030,0.180,12.526,0.000,...,-1.9816,0.6485,2.31,0.416,11.098,2.718,9.116,0.098,0.085,0.413
2953,COP,2019-12-31,0.190,0.188,7.566,4.714,0.130,0.444,6.459,0.221,...,-1.9816,0.6485,2.55,0.090,3.964,0.919,5.440,0.061,0.047,1.103
876,AIG,2019-12-31,0.023,0.054,11.273,-37.158,0.066,0.236,7.292,0.067,...,-1.9816,0.6485,2.20,0.616,13.611,1.387,10.016,0.102,0.097,0.770
8520,OXY,2019-12-31,0.129,-0.023,-26.836,3.974,0.081,0.540,8.755,-0.032,...,-1.9816,0.6485,2.55,0.090,3.964,0.919,5.440,0.061,0.047,1.103


In [64]:
# Create a copy of the test results
results = test_df[['datadate', 'tic', 'next_prccq_change', 'next_relative_quarterly_return']].copy()

# Define columns to drop
drop_cols = ['datadate', 'tic', 'next_prccq_change', 'FFI10_desc']

# Drop the specified columns from train, validation, and test datasets
train_df = train_df.drop(columns=drop_cols)
valid_df = valid_df.drop(columns=drop_cols)
test_df  = test_df.drop(columns=drop_cols)

# Display the first few rows of the training dataset
train_df.head()


,roa,roe,pe_inc,pcf,roce,gpm,evm,npm,de_ratio,ptb,...,r_roe,r_eg,divyield_Median,PEG_trailing_Median,pe_inc_Median,ptb_Median,evm_Median,roce_Median,roe_Median,bm_Median
1335,0.145,0.093,17.928,8.009,0.117,0.507,7.570,0.057,1.440,1.387,...,-6.2954,-10.1619,3.51,0.908,14.712,1.435,9.151,0.102,0.087,0.612
3443,0.204,0.115,41.576,41.821,0.209,0.309,13.130,0.130,0.227,3.479,...,-6.2954,-10.1619,2.95,0.855,12.418,1.368,7.375,0.187,0.106,0.655
11213,0.112,0.094,14.020,16.465,0.122,0.276,7.442,0.013,1.215,1.280,...,-6.2954,-10.1619,3.70,1.084,12.710,1.218,7.059,0.166,0.091,0.689
7019,0.248,0.441,30.332,25.417,0.283,0.386,16.650,0.091,1.872,9.436,...,-6.2954,-10.1619,2.95,0.855,12.418,1.368,7.375,0.187,0.106,0.655
3444,0.204,0.105,47.375,56.869,0.209,0.309,13.130,0.135,0.227,4.730,...,5.6547,6.2280,2.71,1.046,13.920,1.415,7.319,0.183,0.115,0.673


In [65]:
# Define target variable (y) and features (X) for train, validation, and test sets
y_train = train_df['next_relative_quarterly_return'].copy()
X_train = train_df.drop(columns=['next_relative_quarterly_return'])

y_valid = valid_df['next_relative_quarterly_return'].copy()
X_valid = valid_df.drop(columns=['next_relative_quarterly_return'])

y_test  = test_df['next_relative_quarterly_return'].copy()
X_test  = test_df.drop(columns=['next_relative_quarterly_return'])

# Display the first few rows of X_train
X_train.head()


,roa,roe,pe_inc,pcf,roce,gpm,evm,npm,de_ratio,ptb,...,r_roe,r_eg,divyield_Median,PEG_trailing_Median,pe_inc_Median,ptb_Median,evm_Median,roce_Median,roe_Median,bm_Median
1335,0.145,0.093,17.928,8.009,0.117,0.507,7.570,0.057,1.440,1.387,...,-6.2954,-10.1619,3.51,0.908,14.712,1.435,9.151,0.102,0.087,0.612
3443,0.204,0.115,41.576,41.821,0.209,0.309,13.130,0.130,0.227,3.479,...,-6.2954,-10.1619,2.95,0.855,12.418,1.368,7.375,0.187,0.106,0.655
11213,0.112,0.094,14.020,16.465,0.122,0.276,7.442,0.013,1.215,1.280,...,-6.2954,-10.1619,3.70,1.084,12.710,1.218,7.059,0.166,0.091,0.689
7019,0.248,0.441,30.332,25.417,0.283,0.386,16.650,0.091,1.872,9.436,...,-6.2954,-10.1619,2.95,0.855,12.418,1.368,7.375,0.187,0.106,0.655
3444,0.204,0.105,47.375,56.869,0.209,0.309,13.130,0.135,0.227,4.730,...,5.6547,6.2280,2.71,1.046,13.920,1.415,7.319,0.183,0.115,0.673


In [66]:
# Scaling
scaler = MinMaxScaler()

# Fit the scaler on the training/validation data
X_train_scaled = scaler.fit_transform(X_train)

# Now apply the same scaling to the val/test data using the already fitted scaler
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# Convert the scaled data back to DataFrame for consistency
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_valid_scaled = pd.DataFrame(X_valid_scaled, columns=X_valid.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

#X_train_scaled.info()

print(X_train_scaled.columns.tolist())


['roa', 'roe', 'pe_inc', 'pcf', 'roce', 'gpm', 'evm', 'npm', 'de_ratio', 'ptb', 'market_cap', 'prccq_change', 'roa_change', 'roe_change', 'pe_inc_change', 'pcf_change', 'roce_change', 'gpm_change', 'evm_change', 'npm_change', 'de_ratio_change', 'ptb_change', 'relative_quarterly_return', 'volatility_score', 'mktrf', 'smb', 'hml', 'rmw', 'cma', 'rf', 'umd', 'h_1_x', 'h_3_x', 'h_12_x', 'h_1_y', 'h_3_y', 'h_12_y', 'r_f', 'r_mkt', 'r_me', 'r_ia', 'r_roe', 'r_eg', 'divyield_Median', 'PEG_trailing_Median', 'pe_inc_Median', 'ptb_Median', 'evm_Median', 'roce_Median', 'roe_Median', 'bm_Median']


In [67]:
X_train_valid = np.vstack([X_train_scaled, X_valid_scaled])
y_train_valid = np.hstack([y_train, y_valid])


In [68]:
# Define the Ridge model
ridge_model = Ridge()

# Define the parameter grid for alpha values
param_grid = {'alpha': [0.0, 1, 5, 10.0]}

# Perform GridSearchCV to find the best alpha
grid_search = GridSearchCV(ridge_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_valid_scaled, y_valid)

# Get the best alpha
best_alpha = grid_search.best_params_['alpha']
#print(f"Best alpha: {best_alpha}")
# Train the Ridge model with the best alpha
ridge_model = Ridge(alpha=best_alpha)
ridge_model.fit(X_train_valid, y_train_valid)

Ridge(alpha=10.0)

In [69]:

#context = ['relative_quarterly_return_-1', 'relative_quarterly_return_-2', 'relative_quarterly_return_-3', 'mktrf', 'smb', 'hml', 'rmw', 'cma', 'rf', 'umd', 'h_1_x', 'h_3_x', 'h_12_x', 'h_1_y', 'h_3_y', 'h_12_y', 'r_f', 'r_mkt', 'r_me', 'r_ia', 'r_roe', 'r_eg', 'indret_ew', 'CAPEI_Median', 'bm_Median', 'evm_Median', 'pe_inc_Median', 'ptb_Median', 'PEG_trailing_Median', 'divyield_Median', 'roa_Median', 'roce_Median', 'roe_Median', 'gpm_Median', 'npm_Median', 'opmad_Median', 'de_ratio_Median', 'debt_ebitda_Median', 'debt_assets_Median', 'intcov_ratio_Median', 'cash_ratio_Median', 'curr_ratio_Median', 'quick_ratio_Median', 'at_turn_Median', 'inv_turn_Median', 'rect_turn_Median']
context = ['market_cap', 'volatility_score', 'mktrf', 'smb', 'hml', 'rmw', 'cma', 'rf', 'umd', 'h_1_x', 'h_3_x', 'h_12_x', 'h_1_y', 'h_3_y', 'h_12_y', 'r_f', 'r_mkt', 'r_me', 'r_ia', 'r_roe', 'r_eg', 'divyield_Median', 'PEG_trailing_Median', 'pe_inc_Median', 'ptb_Median', 'evm_Median', 'roce_Median', 'roe_Median', 'bm_Median']


C_train = X_train_scaled[context].to_numpy()
C_valid = X_valid_scaled[context].to_numpy()
C_test = X_test_scaled[context].to_numpy()

X_train_context = X_train_scaled.drop(columns=context)
X_valid_context = X_valid_scaled.drop(columns=context)
X_test_context = X_test_scaled.drop(columns=context)

X_train_valid_c = np.vstack([X_train_context, X_valid_context])

In [70]:
# Define the Ridge model
agnostic_model = Ridge()

# Define the parameter grid for alpha values
param_grid = {'alpha': [0.0, 1, 5, 10.0]}

# Perform GridSearchCV to find the best alpha
grid_search = GridSearchCV(ridge_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_valid_scaled, y_valid)

# Get the best alpha
best_alpha = grid_search.best_params_['alpha']
#print(f"Best alpha: {best_alpha}")

# Train the Ridge model with the best alpha
agnostic_model = Ridge(alpha=best_alpha)
agnostic_model.fit(X_train_valid_c, y_train_valid)

Ridge(alpha=10.0)

In [71]:
'''contextualised_model = ContextualizedRegressor()
contextualised_model.fit(C_train, X_train_context.to_numpy(), y_train.to_numpy())'''


'contextualised_model = ContextualizedRegressor()\ncontextualised_model.fit(C_train, X_train_context.to_numpy(), y_train.to_numpy())'

In [72]:
from contextualized.utils import save, load

contextualised_model = load('/Users/edocampione/Desktop/Meng Engineering Science/4YP/models/thecontextualized_model_1.pt' )

#save(contextualised_model, path='thecontextualized_model_1.pt')

In [73]:
y_train_pred3 = agnostic_model.predict(X_train_context.to_numpy())
y_valid_pred3 = agnostic_model.predict(X_valid_context.to_numpy())
y_test_pred3 = agnostic_model.predict(X_test_context.to_numpy())

# Classifier
y_train_pred_binary3 = (y_train_pred3 >= 0).astype(int)
y_valid_pred_binary3 = (y_valid_pred3 >= 0).astype(int)
y_test_pred_binary3 = (y_test_pred3 >= 0).astype(int)

y_train_binary3 = (y_train >= 0).astype(int)
y_valid_binary3 = (y_valid >= 0).astype(int)
y_test_binary3 = (y_test >= 0).astype(int)

print(y_test_pred3)

print(classification_report(y_test_binary3, y_test_pred_binary3))

[ 0.10308967  0.0803166   0.06840921 ...  0.00202876 -0.0030014
  0.00761638]
              precision    recall  f1-score   support

           0       0.58      0.02      0.03       954
           1       0.51      0.99      0.67       986

    accuracy                           0.51      1940
   macro avg       0.54      0.50      0.35      1940
weighted avg       0.54      0.51      0.36      1940



In [74]:
y_train_pred1 = ridge_model.predict(X_train_scaled)
y_valid_pred1 = ridge_model.predict(X_valid_scaled)
y_test_pred1 = ridge_model.predict(X_test_scaled)

# Classifier
y_train_pred_binary1 = (y_train_pred1 >= 0).astype(int)
y_valid_pred_binary1 = (y_valid_pred1 >= 0).astype(int)
y_test_pred_binary1 = (y_test_pred1 >= 0).astype(int)

y_train_binary1 = (y_train >= 0).astype(int)
y_valid_binary1 = (y_valid >= 0).astype(int)
y_test_binary1 = (y_test >= 0).astype(int)

print(y_test_pred1)

print(classification_report(y_test_binary1, y_test_pred_binary1))



[ 0.08587085  0.07925118  0.07679392 ... -0.00610488 -0.02011233
  0.00133202]
              precision    recall  f1-score   support

           0       0.60      0.53      0.56       954
           1       0.59      0.66      0.62       986

    accuracy                           0.59      1940
   macro avg       0.60      0.59      0.59      1940
weighted avg       0.60      0.59      0.59      1940



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but Ridge was fitted without feature names

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but Ridge was fitted without feature names

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but Ridge was fitted without feature names



In [75]:
y_train_pred2 = contextualised_model.predict(C_train, X_train_context.to_numpy())
y_valid_pred2 = contextualised_model.predict(C_valid, X_valid_context.to_numpy())
y_test_pred2 = contextualised_model.predict(C_test, X_test_context.to_numpy())

# Classifier
y_train_pred_binary2 = (y_train_pred2 >= 0).astype(int)
y_valid_pred_binary2 = (y_valid_pred2 >= 0).astype(int)
y_test_pred_binary2 = (y_test_pred2 >= 0).astype(int)

y_train_binary2 = (y_train >= 0).astype(int)
y_valid_binary2 = (y_valid >= 0).astype(int)
y_test_binary2 = (y_test >= 0).astype(int)

print(y_test_pred2)

print(classification_report(y_test_binary2, y_test_pred_binary2))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.



[[ 0.16985063]
 [ 0.16973175]
 [ 0.05471596]
 ...
 [-0.06892331]
 [-0.07395104]
 [-0.06610771]]
              precision    recall  f1-score   support

           0       0.53      0.88      0.66       954
           1       0.68      0.25      0.36       986

    accuracy                           0.56      1940
   macro avg       0.61      0.56      0.51      1940
weighted avg       0.61      0.56      0.51      1940



In [76]:
mse = mean_squared_error(y_test, y_test_pred3)
mae = mean_absolute_error(y_test, y_test_pred3)
print(f"MSE: {mse:.4f}, MAE: {mae:.4f}")

mse = mean_squared_error(y_test, y_test_pred1)
mae = mean_absolute_error(y_test, y_test_pred1)
print(f"MSE: {mse:.4f}, MAE: {mae:.4f}")

mse = mean_squared_error(y_test, y_test_pred2)
mae = mean_absolute_error(y_test, y_test_pred2)
print(f"MSE: {mse:.4f}, MAE: {mae:.4f}")

MSE: 0.0144, MAE: 0.0826
MSE: 0.0140, MAE: 0.0805
MSE: 0.0165, MAE: 0.0838


In [77]:
results['y_pred1'] = y_test_pred1
results['y_pred2'] = y_test_pred2
results['y_pred3'] = y_test_pred3
results1 = results.sort_values(by=['datadate'], ascending=[True])

results1.head(60)



,datadate,tic,next_prccq_change,next_relative_quarterly_return,y_pred1,y_pred2,y_pred3
10615,2013-03-31,TSLA,1.833465,1.799673,0.085871,0.169851,0.103090
7189,2013-03-31,MCD,-0.006921,-0.040714,0.004037,-0.031307,0.020044
9827,2013-03-31,RTX,-0.005245,-0.039037,0.006786,-0.028783,0.019807
3413,2013-03-31,CVX,-0.004040,-0.037832,0.004031,-0.038338,0.012439
8893,2013-03-31,PG,-0.000908,-0.034701,-0.002974,-0.042030,0.016661
4303,2013-03-31,EXPE,0.002249,-0.031543,0.018477,-0.052134,0.014143
11756,2013-03-31,XOM,0.002663,-0.031129,0.001381,-0.044190,0.012163
6145,2013-03-31,KMB,-0.008573,-0.042366,0.019894,-0.003685,0.029914
4759,2013-03-31,GE,0.003028,-0.030765,-0.017536,-0.055266,0.019627
9033,2013-03-31,PGR,0.005936,-0.027857,0.007014,-0.039407,0.014411


In [99]:
def softmax_weighted_return(df, sort_col, target_col, n=5, largest=True):
    if largest:
        selected = df.nlargest(n, sort_col)
    else:
        selected = df.nsmallest(n, sort_col)
    # Compute softmax weights from the predicted scores:
    e_scores = np.exp(selected[sort_col])
    weights = e_scores / e_scores.sum()
    return (selected[target_col] * weights).sum()

def exponential_weighted_return(df, sort_col, target_col, n=5, largest=True, alpha=0.2):
    if largest:
        selected = df.nlargest(n, sort_col)
    else:
        selected = df.nsmallest(n, sort_col)
    selected = selected.sort_values(by=sort_col, ascending=not largest).copy()
    selected['rank'] = range(1, len(selected) + 1)
    # Exponential weights:
    selected['weight'] = np.exp(-alpha * (selected['rank'] - 1))
    selected['weight'] /= selected['weight'].sum()  # Normalize weights
    return (selected[target_col] * selected['weight']).sum()


# Baseline aggregation remains unchanged.
df_avg = results.groupby('datadate').agg(
    stocks=('next_prccq_change', 'size'),
    baseline=('next_prccq_change', 'mean')
).reset_index()

# For the predicted top 10, weight by predicted returns ('pred_rank_score')
df_avg['top_10'] = results.groupby('datadate').apply(
    lambda group: exponential_weighted_return(
        group,
        sort_col='y_pred1',
        target_col='next_prccq_change',
        largest=True
    )
).reset_index(level=0, drop=True)

# For the predicted top 10, weight by predicted returns ('pred_rank_score')
df_avg['top_10_agnostic'] = results.groupby('datadate').apply(
    lambda group: exponential_weighted_return(
        group,
        sort_col='y_pred3',
        target_col='next_prccq_change',
        largest=True
    )
).reset_index(level=0, drop=True)

# For the predicted top 10, weight by predicted returns ('pred_rank_score')
df_avg['top_10_context'] = results.groupby('datadate').apply(
    lambda group: exponential_weighted_return(
        group,
        sort_col='y_pred2',
        target_col='next_prccq_change',
        largest=True
    )
).reset_index(level=0, drop=True)

# For the ideal top 10, weight by the actual returns
df_avg['top_10_ideal'] = results.groupby('datadate').apply(
    lambda group: exponential_weighted_return(
        group,
        sort_col='next_prccq_change',
        target_col='next_prccq_change',
        largest=True
    )
).reset_index(level=0, drop=True)

# For the predicted bottom 10, weight by predicted returns
df_avg['bottom_10'] = results.groupby('datadate').apply(
    lambda group: exponential_weighted_return(
        group,
        sort_col='y_pred1',
        target_col='next_prccq_change',
        largest=False
    )
).reset_index(level=0, drop=True)

# For the predicted bottom 10, weight by predicted returns
df_avg['bottom_10_agnostic'] = results.groupby('datadate').apply(
    lambda group: exponential_weighted_return(
        group,
        sort_col='y_pred3',
        target_col='next_prccq_change',
        largest=False
    )
).reset_index(level=0, drop=True)

# For the predicted bottom 10, weight by predicted returns
df_avg['bottom_10_context'] = results.groupby('datadate').apply(
    lambda group: exponential_weighted_return(
        group,
        sort_col='y_pred2',
        target_col='next_prccq_change',
        largest=False
    )
).reset_index(level=0, drop=True)

# For the ideal bottom 10, weight by the actual returns
df_avg['bottom_10_ideal'] = results.groupby('datadate').apply(
    lambda group: exponential_weighted_return(
        group,
        sort_col='next_prccq_change',
        target_col='next_prccq_change',
        largest=False
    )
).reset_index(level=0, drop=True)

# Display the first 100 rows of the new dataframe
df_avg.head(1)


,datadate,stocks,baseline,top_10,top_10_agnostic,top_10_context,top_10_ideal,bottom_10,bottom_10_agnostic,bottom_10_context,bottom_10_ideal
0,2013-03-31,70,0.077383,0.620756,0.700958,0.577543,0.786961,-0.034844,-0.092305,-0.068183,-0.142434


In [101]:

risk_free_rate = 0.02  # 2% annual risk-free rate, approximated quarterly

for portfolio in ['top_10_agnostic', 'top_10', 'top_10_context', 'top_10_ideal']:
    
    returns = df_avg[portfolio]
    mean_return = returns.mean()*100
    std_return = returns.std()
    
    # Compute excess return
    excess_return = returns.mean() - risk_free_rate

    # Avoid division by zero
    sharpe_ratio = excess_return / std_return if std_return > 0 else np.nan
    
    print(f"Mean quarterly return of the {portfolio} portfolio: {mean_return:.4f}")
    #print(f"Downside deviation of the {portfolio} portfolio: {downside_deviation:.4f}")
    print(f"Sharpe Ratio of the {portfolio} portfolio: {sharpe_ratio:.4f}")
    print("")


for portfolio in ['bottom_10_agnostic', 'bottom_10', 'bottom_10_context', 'bottom_10_ideal']:
    
    returns = df_avg[portfolio]
    mean_return = returns.mean()*100
    std_return = returns.std()
    
    # Compute excess return
    excess_return = returns.mean() + risk_free_rate

    # Avoid division by zero
    sharpe_ratio = np.abs(excess_return / std_return) if std_return > 0 else np.nan
    
    print(f"Mean quarterly return of the {portfolio} portfolio: {mean_return:.4f}")
    #print(f"Downside deviation of the {portfolio} portfolio: {downside_deviation:.4f}")
    print(f"Sharpe Ratio of the {portfolio} portfolio: {sharpe_ratio:.4f}")
    print("")

Mean quarterly return of the top_10_agnostic portfolio: 10.3468
Sharpe Ratio of the top_10_agnostic portfolio: 0.5222

Mean quarterly return of the top_10 portfolio: 10.9716
Sharpe Ratio of the top_10 portfolio: 0.5719

Mean quarterly return of the top_10_context portfolio: 11.5746
Sharpe Ratio of the top_10_context portfolio: 0.6289

Mean quarterly return of the top_10_ideal portfolio: 30.1974
Sharpe Ratio of the top_10_ideal portfolio: 1.8717

Mean quarterly return of the bottom_10_agnostic portfolio: -6.6454
Sharpe Ratio of the bottom_10_agnostic portfolio: 0.5144

Mean quarterly return of the bottom_10 portfolio: -3.0662
Sharpe Ratio of the bottom_10 portfolio: 0.1475

Mean quarterly return of the bottom_10_context portfolio: -7.5897
Sharpe Ratio of the bottom_10_context portfolio: 0.6707

Mean quarterly return of the bottom_10_ideal portfolio: -17.8061
Sharpe Ratio of the bottom_10_ideal portfolio: 1.6045



In [92]:
import plotly.graph_objects as go

# Assuming you have a dataframe 'df_avg' with columns 'datadate', 'baseline_portfolio', and 'top_10'

# Create initial row
initial_row = {
    'datadate': pd.to_datetime('2012-12-31'),
    'baseline': 0,
    'top_10_agnostic': 0,
    'top_10': 0,
    'top_10_context': 0,
    'top_10_ideal': 0,
    'bottom_10_agnostic': 0,
    'bottom_10': 0,
    'bottom_10_context': 0,
    'bottom_10_ideal': 0,
}

# Convert to DataFrame and concatenate
initial_df = pd.DataFrame([initial_row])
df_avg = pd.concat([initial_df, df_avg], ignore_index=True)

# Recalculate cumulative returns from scratch
for col in ['baseline', 'top_10_agnostic', 'top_10', 'top_10_context', 'top_10_ideal']:
    df_avg[f'{col}_cum_return'] = (1 + df_avg[col]).cumprod()
    df_avg[f'{col}_value'] = 1000 * df_avg[f'{col}_cum_return']


# Calculate the cumulative return for both portfolios
df_avg['baseline_cum_return'] = (1 + df_avg['baseline']).cumprod()
df_avg['top_10_a_cum_return'] = (1 + df_avg['top_10_agnostic']).cumprod()
df_avg['top_10_cum_return'] = (1 + df_avg['top_10']).cumprod()
df_avg['top_10_context_cum_return'] = (1 + df_avg['top_10_context']).cumprod()
df_avg['top_10_ideal_cum_return'] = (1 + df_avg['top_10_ideal']).cumprod()

# Calculate the value over time of a $1,000 investment
initial_investment = 1000
df_avg['baseline_value'] = initial_investment * df_avg['baseline_cum_return']
df_avg['top_10_a_value'] = initial_investment * df_avg['top_10_a_cum_return']
df_avg['top_10_value'] = initial_investment * df_avg['top_10_cum_return']
df_avg['top_10_context_value'] = initial_investment * df_avg['top_10_context_cum_return']
df_avg['top_10_ideal_value'] = initial_investment * df_avg['top_10_ideal_cum_return']

# Create the plot
fig = go.Figure()

# Add baseline portfolio trace
fig.add_trace(go.Scatter(x=df_avg['datadate'], y=df_avg['baseline_value'],
                         mode='lines', name='Universe'))

# Add baseline portfolio trace
fig.add_trace(go.Scatter(x=df_avg['datadate'], y=df_avg['top_10_a_value'],
                         mode='lines', name='Context-agnostic'))

# Add top_10 portfolio trace
fig.add_trace(go.Scatter(x=df_avg['datadate'], y=df_avg['top_10_value'],
                         mode='lines', name='Context-aware'))

fig.add_trace(go.Scatter(x=df_avg['datadate'], y=df_avg['top_10_context_value'],
                         mode='lines', name='ContextualisedML'))

# Update the layout
fig.update_layout(
    title='Portfolio Value Over Time ($1,000 Investment)',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
    width=700,   # make the plot narrower
    height=500   # optionally make it taller

)

# Show the plot
fig.show()


In [81]:
import plotly.graph_objects as go

df_avg['baseline_cum_return'] = (1 + df_avg['baseline']).cumprod()
# Recalculate cumulative returns from scratch
for col in ['bottom_10_agnostic', 'bottom_10', 'bottom_10_context', 'bottom_10_ideal']:
    df_avg[f'{col}_cum_return'] = (1 - df_avg[col]).cumprod()
    df_avg[f'{col}_value'] = 1000 * df_avg[f'{col}_cum_return']

df_avg.head(60)

,datadate,baseline,top_10_agnostic,top_10,top_10_context,top_10_ideal,bottom_10_agnostic,bottom_10,bottom_10_context,bottom_10_ideal,...,top_10_a_cum_return,top_10_a_value,bottom_10_agnostic_cum_return,bottom_10_agnostic_value,bottom_10_cum_return,bottom_10_value,bottom_10_context_cum_return,bottom_10_context_value,bottom_10_ideal_cum_return,bottom_10_ideal_value
0,2012-12-31 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1000.000000,1.000000,1000.000000,1.000000,1000.000000,1.000000,1000.000000,1.000000,1000.000000
1,2013-03-31,0.077383,0.851703,0.597240,0.582074,0.953670,-0.103366,-0.035146,-0.084042,-0.147613,...,1.851703,1851.703441,1.103366,1103.366493,1.035146,1035.145790,1.084042,1084.041777,1.147613,1147.613298
2,2013-06-30,0.081841,0.288458,0.461516,0.533611,0.726912,-0.035253,0.006475,-0.070096,-0.093170,...,2.385843,2385.842658,1.142263,1142.263246,1.028443,1028.442887,1.160029,1160.028601,1.254536,1254.535980
3,2013-09-30,0.105676,0.023985,0.016495,0.193929,0.314477,-0.021761,0.019935,-0.024217,-0.136803,...,2.443068,2443.067610,1.167120,1167.119833,1.007941,1007.940661,1.188121,1188.121223,1.426161,1426.160649
4,2013-12-31,0.030107,0.144947,0.155226,0.163742,0.300947,0.026619,-0.000444,0.015349,-0.121977,...,2.797183,2797.183327,1.136053,1136.052808,1.008388,1008.387858,1.169885,1169.885204,1.600119,1600.119043
5,2014-03-31,0.031233,0.097174,0.152888,0.055536,0.217669,-0.209779,-0.184048,-0.211720,-0.250107,...,3.068997,3068.996593,1.374373,1374.373050,1.193980,1193.979663,1.417574,1417.573743,2.000321,2000.320692
6,2014-06-30,0.021678,0.032145,0.063588,0.024955,0.227391,-0.057937,-0.043879,-0.038903,-0.141557,...,3.167651,3167.650634,1.454000,1453.999942,1.246371,1246.370709,1.472722,1472.722171,2.283481,2283.481063
7,2014-09-30,0.041867,0.006366,-0.039899,0.045459,0.222324,-0.076472,-0.015596,-0.077723,-0.196608,...,3.187815,3187.815346,1.565191,1565.190618,1.265809,1265.809329,1.587187,1587.186651,2.732432,2732.432032
8,2014-12-31,0.013406,-0.027496,-0.014614,-0.028449,0.195806,-0.018090,-0.041130,-0.000464,-0.152838,...,3.100164,3100.163729,1.593505,1593.505168,1.317872,1317.871686,1.587922,1587.922454,3.150053,3150.052839
9,2015-03-31,0.018974,-0.035611,-0.007613,0.022564,0.383962,-0.054962,-0.001983,-0.038922,-0.172164,...,2.989763,2989.762726,1.681088,1681.088119,1.320486,1320.485634,1.649727,1649.727122,3.692379,3692.378728


In [82]:
# Calculate the value over time of a $1,000 investment
initial_investment = 1000
df_avg['baseline_value'] = initial_investment * df_avg['baseline_cum_return']
df_avg['bottom_10_agnostic_value'] = initial_investment * df_avg['bottom_10_agnostic_cum_return']
df_avg['bottom_10_value'] = initial_investment * df_avg['bottom_10_cum_return']
df_avg['bottom_10_context_value'] = initial_investment * df_avg['bottom_10_context_cum_return']
df_avg['bottom_10_ideal_value'] = initial_investment * df_avg['bottom_10_ideal_cum_return']

# Create the plot
fig = go.Figure()

# Add baseline portfolio trace
fig.add_trace(go.Scatter(x=df_avg['datadate'], y=df_avg['baseline_value'],
                         mode='lines', name='Universe'))

# Add baseline portfolio trace
fig.add_trace(go.Scatter(x=df_avg['datadate'], y=df_avg['bottom_10_agnostic_value'],
                         mode='lines', name='Context-agnostic'))

# Add top_10 portfolio trace
fig.add_trace(go.Scatter(x=df_avg['datadate'], y=df_avg['bottom_10_value'],
                         mode='lines', name='Context-aware'))

fig.add_trace(go.Scatter(x=df_avg['datadate'], y=df_avg['bottom_10_context_value'],
                         mode='lines', name='ContextualisedML'))

# Update the layout
fig.update_layout(
    title='Portfolio Value Over Time ($1,000 Investment)',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
    width=700,   # make the plot narrower
    height=500   # optionally make it taller

)

# Show the plot
fig.show()
